In [ ]:

# DESCRIPTION
# This notebook receives an input file with an "ID" column representing the company LinkedIn IDs or a list of company domains that we want to scrape for.
# It uses the provided LinkedIn cookie to scrape Sales Nav for the target persona and calls BetterContact to find emails.
import csv
import requests
import json
import time
import re
import random
import ipywidgets as widgets
from IPython.display import display
import pandas as pd
import urllib.parse
import http.client
from bs4 import BeautifulSoup
import concurrent.futures

# ENV Variables
SERPER_API_KEY = '<API_KEY>'
ZENROWS_API_KEY = '<API_KEY>'
DOMAIN_COLUMN = 'domain' # Column where domain name is found



# Function to decode the entire URL-encoded query string
def extract_li_query(full_url):
    parsed_url = urllib.parse.urlparse(full_url)
    query_params = urllib.parse.parse_qs(parsed_url.query)
    query_value = query_params.get('query', [None])[0]
    return urllib.parse.unquote(query_value)

# Function to encode "text" values back
def encode_text_values(match):
    text_value = match.group(1)
    return f"text:{urllib.parse.quote(text_value)}"

# Function to encode URN substring
def url_encode_urn_substring(input_string):
    pattern = r'(urn:[^,]*)(,?)'
    def encode_match(match):
        urn_part = match.group(1)
        encoded_urn = urllib.parse.quote(urn_part)
        return encoded_urn + match.group(2)
    return re.sub(pattern, encode_match, input_string)

# Function to find the LinkedIn URL
def find_linkedin_url(company_name):
    conn = http.client.HTTPSConnection("google.serper.dev")
    payload = json.dumps({
        "q": f"{company_name} site:linkedin.com/company"
    })
    headers = {
        'X-API-KEY': SERPER_API_KEY,
        'Content-Type': 'application/json'
    }
    conn.request("POST", "/search", payload, headers)
    res = conn.getresponse()
    data = res.read().decode("utf-8")
    results = json.loads(data)
    if not results or not results.get('organic'):
        return None
    website = results['organic'][0]['link']
    return website

# Function to extract LinkedIn company ID
def extract_linkedin_company_id(linkedin_url):
    print(f"Fetching {linkedin_url}")
    try:
        params = {
            'url': linkedin_url,
            'apikey': ZENROWS_API_KEY,
            'js_render': 'true',
            'custom_headers': 'true',
            'premium_proxy': 'true',
            'wait_for': '[data-entity-id]'
        }
        headers = {
            'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36',
        }

        retries = 4
        backoff_factor = 2
        initial_wait = 2  # initial wait time in seconds

        for attempt in range(retries):
            response = requests.get('https://api.zenrows.com/v1/', params=params, headers=headers)

            if response.status_code == 200:
                soup = BeautifulSoup(response.content, 'html.parser')

                # Use regex to find the number in the format urn:li:organization:<number>
                linkedin_company_id_match = re.search(r'urn:li:organization:(\d+)', str(soup))

                if linkedin_company_id_match:
                    return linkedin_company_id_match.group(1)

            print(f"Error crawling {linkedin_url}: {response.text}, attempt {attempt + 1} of {retries}")
            wait_time = initial_wait * (backoff_factor ** attempt)
            time.sleep(wait_time)  # Exponential backoff
    except Exception as e:
        print(f"Exception occurred: {e}")
        return None
    return None

# Function to process a single company's domain
def get_company_linkedin_data(company_domain):
    linkedin_url = find_linkedin_url(company_domain)
    if not linkedin_url:
        return {
            'domain': company_domain,
            'linkedin_company_id': None,
            'linkedin_company_urn': None,
            'linkedin_company_url': None
        }
    linkedin_company_id = extract_linkedin_company_id(linkedin_url)
    if linkedin_company_id:
        linkedin_company_urn = f'(id%3Aurn%253Ali%253Aorganization%253A{linkedin_company_id}%2CselectionType%3AINCLUDED)'
        linkedin_company_url = f'https://linkedin.com/company/{linkedin_company_id}'
    else:
        linkedin_company_urn = None
        linkedin_company_url = None

    return {
        'domain': company_domain,
        'linkedin_company_id': linkedin_company_id,
        'linkedin_company_urn': linkedin_company_urn,
        'linkedin_company_url': linkedin_company_url
    }


# LinkedIn API request function with progress bar and error handling
def make_linkedin_request(li_query, cookie, csrf_token, total_searches, query_type="salesApiLeadSearch", decoration_id="com.linkedin.sales.deco.desktop.searchv2.LeadSearchResult-14"):
    start = 0
    count = 25
    all_elements = []
    headers = {
        'cookie': cookie,
        'csrf-token': csrf_token,
        'x-restli-protocol-version': '2.0.0'
    }

    while True:
        request_url = f'https://www.linkedin.com/sales-api/{query_type}?q=searchQuery&query={li_query}&start={start}&count={count}&decorationId={decoration_id}'

        response = requests.get(request_url, headers=headers)

        if response.status_code != 200:
            print(f"Error: Received status code {response.status_code} for URL: {request_url}")
            break

        response_data = response.json()
        elements = response_data.get('elements', [])
        all_elements.extend(elements)
        paging = response_data.get('paging', {})
        total = int(paging.get('total', 0))
        start += count
        total_searches += 1

        if start >= total or total_searches >= 1:
            break

    return all_elements, total_searches

# Function to load CSV and process IDs or domains.
def load_and_process_csv(file_path, list_type, domain_column_name):

    df = pd.read_csv(file_path, dtype={'ID': str})

    if list_type == 'Domain':

      # Using ThreadPoolExecutor to run 10 concurrent rows at a time
      with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
          results = list(executor.map(get_company_linkedin_data, df[domain_column_name]))

      # Convert the results into a DataFrame
      results_df = pd.DataFrame(results)

      # Drop rows with missing LinkedIn company ID and remove duplicates
      results_df = results_df.dropna(subset=['linkedin_company_urn']).drop_duplicates(subset=['linkedin_company_urn'])

      return results_df['linkedin_company_urn'].tolist()

    else:
      df = df.dropna(subset=['ID']).drop_duplicates(subset=['ID'])

      return df["ID"].apply(lambda linkedin_id: f'(id%3Aurn%253Ali%253Aorganization%253A{linkedin_id}%2CselectionType%3AINCLUDED)').tolist()

# Function to group list into batches
def group_into_batches(lst, batch_size=20):
    for i in range(0, len(lst), batch_size):
        yield lst[i:i+batch_size]

# Function to randomly sleep for rate-limiting
def sleep_random_time():
    time.sleep(2 * random.random())

# BetterContact API request function for bulk data
def call_better_contact_api_bulk(data_list):
    url = "https://app.bettercontact.rocks/api/v2/async"
    querystring = {"api_key":"<API_KEY>"}

    payload = {
        "data": data_list,
        "webhook": "https://hooks.zapier.com/hooks/catch/9883452/251gst6/",
        "verify_catch_all": True,
        "enrich_phone_number": False,
    }
    headers = {"Content-Type": "application/json"}


    response = requests.request("POST", url, json=payload, headers=headers, params=querystring)

    if response.status_code == 201:
        print(f"Success: {response.json()}")
    else:
        print(f"Error: {response.status_code}, {response.text}")

# Base URLs for different query types
query_base_urls = {
    "Founders": "https://www.linkedin.com/sales/search/people?query=(recentSearchParam%3A(id%3A3126161513%2CdoLogHistory%3Atrue)%2Cfilters%3AList((type%3ACURRENT_TITLE%2Cvalues%3AList((id%3A8%2Ctext%3AChief%2520Executive%2520Officer%2CselectionType%3AINCLUDED)%2C(id%3A35%2Ctext%3AFounder%2CselectionType%3AINCLUDED)%2C(id%3A103%2Ctext%3ACo-Founder%2CselectionType%3AINCLUDED)))%2C(type%3AREGION%2Cvalues%3AList((id%3A103644278%2Ctext%3AUnited%2520States%2CselectionType%3AINCLUDED)%2C(id%3A101165590%2Ctext%3AUnited%2520Kingdom%2CselectionType%3AINCLUDED)%2C(id%3A105646813%2Ctext%3ASpain%2CselectionType%3AINCLUDED)%2C(id%3A100456013%2Ctext%3AFinland%2CselectionType%3AINCLUDED)%2C(id%3A105117694%2Ctext%3ASweden%2CselectionType%3AINCLUDED)%2C(id%3A100565514%2Ctext%3ABelgium%2CselectionType%3AINCLUDED)%2C(id%3A102890719%2Ctext%3ANetherlands%2CselectionType%3AINCLUDED)%2C(id%3A104042105%2Ctext%3ALuxembourg%2CselectionType%3AINCLUDED)%2C(id%3A101174742%2Ctext%3ACanada%2CselectionType%3AINCLUDED)))%2C(type%3ACURRENT_COMPANY%2Cvalues%3AList({list}))))&sessionId=T26sTufNShK1Bdm%2BATCbtw%3D%3D",
    "Finance": "https://www.linkedin.com/sales/search/people?query=(recentSearchParam%3A(id%3A3365498601%2CdoLogHistory%3Atrue)%2Cfilters%3AList((type%3AFUNCTION%2Cvalues%3AList((id%3A26%2Ctext%3AFinance%2CselectionType%3AINCLUDED)))%2C(type%3ASENIORITY_LEVEL%2Cvalues%3AList((id%3A220%2Ctext%3ADirector%2CselectionType%3AINCLUDED)%2C(id%3A200%2Ctext%3AEntry%2520Level%2520Manager%2CselectionType%3AINCLUDED)%2C(id%3A130%2Ctext%3AStrategic%2CselectionType%3AINCLUDED)%2C(id%3A210%2Ctext%3AExperienced%2520Manager%2CselectionType%3AINCLUDED)%2C(id%3A310%2Ctext%3ACXO%2CselectionType%3AINCLUDED)%2C(id%3A300%2Ctext%3AVice%2520President%2CselectionType%3AINCLUDED)%2C(id%3A320%2Ctext%3AOwner%2520%252F%2520Partner%2CselectionType%3AINCLUDED)))%2C(type%3AREGION%2Cvalues%3AList((id%3A103644278%2Ctext%3AUnited%2520States%2CselectionType%3AINCLUDED)%2C(id%3A101165590%2Ctext%3AUnited%2520Kingdom%2CselectionType%3AINCLUDED)%2C(id%3A101174742%2Ctext%3ACanada%2CselectionType%3AINCLUDED)%2C(id%3A102890719%2Ctext%3ANetherlands%2CselectionType%3AINCLUDED)%2C(id%3A105646813%2Ctext%3ASpain%2CselectionType%3AINCLUDED)))%2C(type%3ACURRENT_COMPANY%2Cvalues%3AList({list}))))&sessionId=53xZKPF3TB6JlmpsRw2THw%3D%3D",
    "Sales": "https://www.linkedin.com/sales/search/people?query=(recentSearchParam%3A(id%3A3365498601%2CdoLogHistory%3Atrue)%2Cfilters%3AList((type%3AFUNCTION%2Cvalues%3AList((id%3A26%2Ctext%3ASales%2CselectionType%3AINCLUDED)))%2C(type%3ASENIORITY_LEVEL%2Cvalues%3AList((id%3A220%2Ctext%3ADirector%2CselectionType%3AINCLUDED)%2C(id%3A200%2Ctext%3AEntry%2520Level%2520Manager%2CselectionType%3AINCLUDED)%2C(id%3A130%2Ctext%3AStrategic%2CselectionType%3AINCLUDED)%2C(id%3A210%2Ctext%3AExperienced%2520Manager%2CselectionType%3AINCLUDED)%2C(id%3A310%2Ctext%3ACXO%2CselectionType%3AINCLUDED)%2C(id%3A300%2Ctext%3AVice%2520President%2CselectionType%3AINCLUDED)%2C(id%3A320%2Ctext%3AOwner%2520%252F%2520Partner%2CselectionType%3AINCLUDED)))%2C(type%3AREGION%2Cvalues%3AList((id%3A103644278%2Ctext%3AUnited%2520States%2CselectionType%3AINCLUDED)%2C(id%3A101165590%2Ctext%3AUnited%2520Kingdom%2CselectionType%3AINCLUDED)%2C(id%3A101174742%2Ctext%3ACanada%2CselectionType%3AINCLUDED)%2C(id%3A102890719%2Ctext%3ANetherlands%2CselectionType%3AINCLUDED)%2C(id%3A105646813%2Ctext%3ASpain%2CselectionType%3AINCLUDED)))%2C(type%3ACURRENT_COMPANY%2Cvalues%3AList({list}))))&sessionId=53xZKPF3TB6JlmpsRw2THw%3D%3D"
    "Customer Success": "https://www.linkedin.com/sales/search/people?query=(recentSearchParam%3A(id%3A3365498601%2CdoLogHistory%3Atrue)%2Cfilters%3AList((type%3AFUNCTION%2Cvalues%3AList((id%3A26%2Ctext%3ACustomer%2520Success%2520and%2520Support%2CselectionType%3AINCLUDED)))%2C(type%3ASENIORITY_LEVEL%2Cvalues%3AList((id%3A220%2Ctext%3ADirector%2CselectionType%3AINCLUDED)%2C(id%3A200%2Ctext%3AEntry%2520Level%2520Manager%2CselectionType%3AINCLUDED)%2C(id%3A130%2Ctext%3AStrategic%2CselectionType%3AINCLUDED)%2C(id%3A210%2Ctext%3AExperienced%2520Manager%2CselectionType%3AINCLUDED)%2C(id%3A310%2Ctext%3ACXO%2CselectionType%3AINCLUDED)%2C(id%3A300%2Ctext%3AVice%2520President%2CselectionType%3AINCLUDED)%2C(id%3A320%2Ctext%3AOwner%2520%252F%2520Partner%2CselectionType%3AINCLUDED)))%2C(type%3AREGION%2Cvalues%3AList((id%3A103644278%2Ctext%3AUnited%2520States%2CselectionType%3AINCLUDED)%2C(id%3A101165590%2Ctext%3AUnited%2520Kingdom%2CselectionType%3AINCLUDED)%2C(id%3A101174742%2Ctext%3ACanada%2CselectionType%3AINCLUDED)%2C(id%3A102890719%2Ctext%3ANetherlands%2CselectionType%3AINCLUDED)%2C(id%3A105646813%2Ctext%3ASpain%2CselectionType%3AINCLUDED)))%2C(type%3ACURRENT_COMPANY%2Cvalues%3AList({list}))))&sessionId=53xZKPF3TB6JlmpsRw2THw%3D%3D"
}



In [ ]:
# This piece just processes the company list and prepares it for the person prospection in the next cell.
strings = []
input_file_input = widgets.Text(description="Input File:", value="domains.csv")
domain_name_column = widgets.Text(description="Domain col:", value="domain")
list_type_options = widgets.Dropdown(
    options=["Domain", "ID"],
    description="List type"
)
run_button = widgets.Button(description="Run Script")
output_area = widgets.Output()

# Display the GUI
display(input_file_input, list_type_options, domain_name_column, run_button, output_area)

# Function to process everything and generate output
def process_data(_):
    global strings
    with output_area:
        output_area.clear_output()
        input_file = input_file_input.value
        list_type = list_type_options.value
        domain_column = domain_name_column.value

        if not input_file or not list_type or not domain_column:
            print("Please fill all fields.")
            return

        strings = load_and_process_csv(input_file, list_type, domain_column)

# Bind button click to processing function
run_button.on_click(process_data)

In [ ]:
# GUI Elements
cookie_input = widgets.Text(description="Cookie:")
csrf_token_input = widgets.Text(description="CSRF Token:")
output_file_input = widgets.Text(description="Output File:", value="output.csv")
query_options = widgets.Dropdown(
    options=["Sales", "Finance", "Founders"],
    description="Query Type:"
)
run_button = widgets.Button(description="Run Script")
output_area = widgets.Output()

# Display the GUI
display(cookie_input, csrf_token_input, output_file_input, query_options, run_button, output_area)

# Function to process everything and generate output
def process_data(_):
    with output_area:
        output_area.clear_output()
        cookie = cookie_input.value
        csrf_token = csrf_token_input.value
        input_file = input_file_input.value
        output_file = output_file_input.value
        query_type = query_options.value
        base_url = query_base_urls[query_type]
        list_type = list_type_options.value

        if not cookie or not csrf_token or not output_file:
            print("Please fill all fields.")
            return
        batches = list(group_into_batches(strings, 20))
        urls = [base_url.replace("{list}", "%2C".join(batch)) for batch in batches]

        # Initialize progress bar
        progress_bar = widgets.IntProgress(value=0, min=0, max=len(urls), description="Progress:")
        display(progress_bar)

        lead_data = []
        total_searches = 0

        for url in urls:
            li_query = re.sub(r'text:([^,)]*)', encode_text_values, extract_li_query(url))
            li_query = url_encode_urn_substring(li_query)
            leads, total_searches = make_linkedin_request(li_query, cookie, csrf_token, total_searches)
            lead_data.extend(leads)

            progress_bar.value += 1  # Update progress bar
            sleep_random_time()

        final_df = pd.DataFrame(lead_data)


        final_df['currentJobTitle'] = final_df['currentPositions'].apply(lambda pos: pos[0]['title'] if pos else None)
        final_df['currentCompanyName'] = final_df['currentPositions'].apply(lambda pos: pos[0]['companyName'] if pos else None)
        final_df['unique_id'] = final_df['entityUrn'].str.extract(r'urn:li:fs_salesProfile:\(([^,]+)')
        final_df['linkedin_url'] = "https://linkedin.com/in/" + final_df['unique_id']
        final_df = final_df.drop(columns=['unique_id'])

        final_df.to_csv(output_file)
        print("Data processing complete. Output saved to:", output_file)

        contact_data_list = final_df.apply(lambda row: {
            "first_name": row.get('firstName', ''),
            "last_name": row.get('lastName', ''),
            "company": row.get('currentCompanyName', ''),
            "linkedin_url": row.get('linkedin_url', ''),
            "custom_fields": {
                "list": 'sales_people',
                "job_title": row.get('currentJobTitle', ''),
            }
        }, axis=1).tolist()

        # Call BetterContact API with all contacts at once
        call_better_contact_api_bulk(contact_data_list)

# Bind button click to processing function
run_button.on_click(process_data)